<a href="https://colab.research.google.com/github/tsa87/semi-jtvae/blob/master/semi_jtvae_train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Mar 25 02:59:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install torch==1.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install dive-into-graphs

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.2 MB/s eta 0:13:48tcmalloc: large alloc 1147494400 bytes == 0x564dde98e000 @  0x7fc7e0a2e615 0x564ddbde817c 0x564ddbec847a 0x564ddbdeaf9d 0x564ddbedcd4d 0x564ddbe5eec8 0x564ddbe59a2e 0x564ddbdec88a 0x564ddbe5ed30 0x564ddbe59a2e 0x564ddbdec88a 0x564ddbe5b719 0x564ddbeddb76 0x564ddbe5ad95 0x564ddbeddb76 0x564ddbe5ad95 0x564ddbeddb76 0x564ddbe5ad95 0x564ddbdecce9 0x564ddbe30579 0x564ddbdeb902 0x564ddbe5ec4d 0x564ddbe59a2e 0x564ddbdec88a 0x564ddbe5b719 0x564ddbe59a2e 0x564ddbdec88a 0x564ddbe5a8f6 0x564ddbdec7aa 0x564ddbe5ab4f 0x564ddbe59a2e
     |██████████████████▌             | 1055.7 MB 1.2 MB/s eta 0:10:25tcmalloc: large alloc 1434370048 bytes == 0x564e22fe4000 @  0x7fc7e0a2e615 0x564ddbde817c 0x564ddbec847a 0x564ddbdeaf9d 0x564ddbedcd4d 0x564ddbe5eec8 0x564ddbe59a2e 0x564ddbdec88a 0x564ddbe5ed30 0x564ddbe59a2e 0x564ddbdec88a 0x564ddbe5b719 0x564ddbeddb76 0x564d

In [ ]:
!git clone https://github.com/tsa87/anb-net
%cd anb-net

Cloning into 'anb-net'...
remote: Enumerating objects: 278, done.
remote: Counting objects: 100% (278/278), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 278 (delta 76), reused 254 (delta 58), pack-reused 0
Receiving objects: 100% (278/278), 8.73 MiB | 25.69 MiB/s, done.
Resolving deltas: 100% (76/76), done.
/content/anb-net


In [ ]:
%load_ext autoreload
%autoreload 2
import os
import json
import argparse

import numpy as np
import torch

from dig.ggraph.dataset import ZINC250k, ZINC800
from molecule_optimizer.externals.fast_jtnn.datautils import SemiMolTreeFolder
from molecule_optimizer.runner.semi_jtvae import SemiJTVAEGeneratorPredictor
from torch_geometric.data import DenseDataLoader

In [ ]:
conf = json.load(open("training/configs/rand_gen_zinc250k_config_dict.json"))

In [ ]:
print("Processing Dataset...")
_ = ZINC250k(
    root=conf["data"]["root"],
    one_shot=False,
    use_aug=False,
)

Processing Dataset...
making raw files: ../data/zinc/raw


Processing...
making processed files: ../data/zinc/zinc250k_property/processed
Done!


In [ ]:
zinc_250_jt = torch.load(
    os.path.join(conf["data"]["root"], conf["data"]["processed_path"])
)
smiles = zinc_250_jt[-1]
labels = zinc_250_jt[0].y

In [ ]:
runner = SemiJTVAEGeneratorPredictor(smiles)

In [ ]:
runner.get_model(
    "rand_gen",
    {
        "hidden_size": conf["model"]["hidden_size"],
        "latent_size": conf["model"]["latent_size"],
        "depthT": conf["model"]["depthT"],
        "depthG": conf["model"]["depthG"],
        "label_size": 1,
        "label_mean": float(torch.mean(labels)),
        "label_var": float(torch.var(labels)),
    },
)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [ ]:
print(type(runner))

<class 'molecule_optimizer.runner.semi_jtvae.SemiJTVAEGeneratorPredictor'>


In [ ]:
import pickle 
filehandler = open('runner.xml', 'wb') 
pickle.dump(runner, filehandler)

In [ ]:
filehandler = open('runner.xml', 'rb') 
runner = pickle.load(filehandler)

In [ ]:
preprocessed, labels = runner.preprocess(smiles, labels)

In [ ]:
loader = SemiMolTreeFolder(
      preprocessed,
      labels,
      runner.vocab,
      conf["batch_size"],
      num_workers=conf["num_workers"],
  )


In [ ]:
print("Training model...")
runner.train_gen_pred(
    loader=loader,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=conf["print_iter"],
    save_iter=conf["save_iter"],
)